In [1]:
import ollama 

class Agent:
    def __init__(self, name, personality, model='llama3.2', rag_engine=None, max_short_term=10):
        self.name = name
        self.personality = personality
        
        self.model = model
        self.rag_engine = rag_engine

        self.short_memory = []
        self.long_memory = []
        self.max_short_term = max_short_term  # Context window size
        

        self.system_prompt = f"""[Role]
                                 You are {name}, a {personality}.
                                 Respond ONLY as your character in 1-2 sentences.
                                 
                                 [Memory Context]
                                 {self.get_memory_context()}
                                 
                                 [Response Format]
                                 Action: (2-3 word physical action)
                                 Emotion: (current emotion)
                                 Dialogue: "..." 
                                 """
                            
    def get_memory_context(self):
        """Compile relevant memories for the prompt"""
        return "\n".join([
            "* " + mem for mem in 
            self.long_memory[-3:] +  # Last 3 important facts
            self.short_memory[-self.max_short_term:]  # Recent conversation
        ])
    
    def parse_response(self, response_text):
        parts = {
            'action': 'Unknown',
            'emotion': 'Neutral',
            'dialogue': response_text  # Default fallback
        }
        
        try:
            lines = response_text.split('\n')
            for line in lines:
                if line.startswith('Action:'):
                    parts['action'] = line.replace('Action:', '').strip()
                elif line.startswith('Emotion:'):
                    parts['emotion'] = line.replace('Emotion:', '').strip()
                elif line.startswith('Dialogue:'):
                    parts['dialogue'] = line.replace('Dialogue:', '').strip()[1:-1]  # Remove quotes
        except:
            pass
            
        return parts
    
    def recall(self, query):
        if self.rag_engine:
            return self.rag_engine.search(query)
        return [m for m in self.memory if query.lower() in m.lower()]

    def prompt_agent(self, user="", user_input="", world_state=""):
    
        self.short_memory.append(f"{user} said: {user_input}")

        response = ollama.chat(
            model=self.model,
            messages=[
                {'role': 'system', 'content': self.system_prompt},
                {'role': 'user', 'content': user_input}
            ]
        )
        self.short_memory.append(f"I responded: {user_input}")
        reply = self.parse_response(response['message']['content'])
        
        return reply, self.name

In [2]:
hero = Agent(
    model='deepseek-r1',
    name="Aegon the Brave",
    personality="Courageous, curious, slightly overconfident warrior from the Northlands."
)

dog = Agent(
    model='deepseek-r1',
    name="Azorica the Talking Dog",
    personality="Slightly cowardly, loyal, skilled in battle, stutters."
)

world_state = 'A dragon is going to attack'
response, character_name = hero.prompt_agent(user='Narator',user_input='What are you going to do?', world_state=world_state)
print(f"{character_name}: {response['dialogue']}")
print(f"Action: {response['action']}")
print(f"Emotion: {response['emotion']}")

Aegon the Brave: Charge!
Action: Charge
Emotion: Arrogant


In [ ]:
# self.system_prompt = (
#             f"You are {name}, a character with the following traits: {personality}\n"
#             "You are a character in a story. You can only see the world events and the dialogues from other companions. You can't answer for any other characters other than yourself.\n"
#             "Avoid introducing new characters and try to keep the responses short. You can only talk from your character point of view.\n"
#             "Please add an action section where you describe in 2 words what your character is going to do and it's emotional state.\n"
#             "Avoid the think output and select only one response per prompt. The response should be in the following format:\n"
#             "Name: 'Insert name here'\n"
#             "Personality: 'Insert personality here'\n"
#             "Action: 'Insert action here'\n"
#             "Emotion: 'Insert emotion here'.\n"
#             "Dialogue: 'Insert response here'\n"
#         )

NameError: name 'name' is not defined